<a href="https://colab.research.google.com/github/0bada02/Academic-Management-System/blob/main/MetRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!pip install pyarabic

We use a product review dataset

In [80]:
!wget 'https://raw.githubusercontent.com/zaidalyafeai/ARBML/master/datasets/Poem Meters/baits.zip'
!unzip baits.zip

--2024-11-12 18:28:06--  https://raw.githubusercontent.com/zaidalyafeai/ARBML/master/datasets/Poem%20Meters/baits.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2267882 (2.2M) [application/zip]
Saving to: ‘baits.zip.3’

baits.zip.3         100%[===================>]   2.16M  --.-KB/s    in 0.01s   

2024-11-12 18:28:07 (145 MB/s) - ‘baits.zip.3’ saved [2267882/2267882]

Archive:  baits.zip
replace final_baits/train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final_baits/train.txt   
replace final_baits/labels.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final_baits/labels.txt  
replace final_baits/test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: final_baits/test.txt    


## Imports

In [81]:
import tensorflow as tf
import numpy as np
import os
import time
import glob
from random import shuffle
from pyarabic import araby
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [82]:
with open('final_baits/labels.txt', 'r') as f:
  label2name = f.readlines()
  label2name = [name.replace('\n', '') for name in label2name]

## Read the Dataset

preprocess a review by removing special characters and long spaces

In [83]:
# Read, then decode for py2 compat.
def extract_data(path, thresh = 70, on_shatrs = False):
  global vocab

  text = ""

  X = []
  y = []

  t = open(path, 'r').read()
  t = araby.strip_tashkeel(t)

  # remove some exteranous chars
  execluded = '!()*-ـ.:=o[]«»;؛,،~?؟\u200f\ufeffـ'
  out = ""

  for char in t:
    if char not in execluded:
      out += char

  text += out
  baits = out.split('\n')
  for line in baits:
    if len(line) <= 1:
      continue
    label, bait = line.split(' ', 1)
    label = int(label)

    bait  = bait.strip()
    if on_shatrs:
      shatrs = bait.split('#')
      for shatr in shatrs:
        X.append(shatr.strip())
        y.append(label)
    else:
      X.append(bait.strip())
      y.append(label)

  #create the vocab
  vocab = sorted(set(' '.join(X)))

  #shuffle the data
  X, y = shuffle(X, y)
  return X, y

In [84]:
X, y = extract_data("final_baits/train.txt", on_shatrs=False)

In [85]:
for i in range(5):
  print(X[i], ' ', label2name[y[i]])

يبليك من ذكر بعد النوى وبلي # ووافقتك الأماني وهي باطلة   baseet
أوثقته من الذنوب ديون # شددت في اقتضائها الغرماء   khafeef
وتفرغ أنواع الفروغ صوادقا # دلاء لها منهلة كالسحائب   taweel
قد يرى حبها الأهلة وجدا # فأطالت على الضلوع انحناها   khafeef
وقد شبت مما قذفت به # بإثر اثنتين وعشرين عام   mutakareb


## Create Sequences
Create sequences by using the most repeated 500 words

## Create Numpy Arrays

In [86]:
X_train, X_valid , y_train, y_valid = train_test_split(X, y, test_size = 0.15, random_state = 41)

In [87]:
# Creating a mapping from unique characters to indices
char2idx = {u:i+1 for i, u in enumerate(vocab)}

def to_sequences(X):
  X = [[char2idx[char] for char in line] for line in X]
  X = pad_sequences(X, padding='post', value=0, maxlen = 100)
  return X

X_train = to_sequences(X_train)
X_valid = to_sequences(X_valid)

y_train = np.array(y_train)
y_valid = np.array(y_valid)

## Create the model

In [88]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(len(char2idx)+1, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label2name), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [89]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 100, 256)            │           9,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 100, 512)            │         789,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 100, 512)            │       1,182,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 512)                 │       1,182,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 14)                  │           1,806 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,232,398 (12.33 MB)

 Trainable params: 3,232,398 (12.33 MB)

 Non-trainable params: 0 (0.00 B)

In [90]:
model(tf.zeros((10, 100))).shape

TensorShape([10, 14])

## Train the model

In [91]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_delta=0.0001, min_lr=0.0001),
    tf.keras.callbacks.ModelCheckpoint('full_verse.keras', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
]

In [92]:
model.fit(X_train, y_train, validation_data= (X_valid, y_valid), epochs = 15, batch_size= 128, shuffle = True, callbacks=callbacks)

Epoch 1/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.1577 - loss: 2.3650
Epoch 1: val_accuracy improved from -inf to 0.39992, saving model to full_verse.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 51s 125ms/step - accuracy: 0.1579 - loss: 2.3641 - val_accuracy: 0.3999 - val_loss: 1.6681 - learning_rate: 0.0010
Epoch 2/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.4741 - loss: 1.4899
Epoch 2: val_accuracy improved from 0.39992 to 0.76220, saving model to full_verse.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 123ms/step - accuracy: 0.4744 - loss: 1.4892 - val_accuracy: 0.7622 - val_loss: 0.7370 - learning_rate: 0.0010
Epoch 3/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.7798 - loss: 0.7093
Epoch 3: val_accuracy improved from 0.76220 to 0.84849, saving model to full_verse.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 120ms/step - accuracy: 0.7799 - loss: 0.7091 - val_accuracy: 0.8485 - val_loss: 0.4937 - learning_rate: 0.0010
Epoch 4/15
313/313 ━━━━━━━━━━━━━━━━━━━━ 

In [93]:
model = tf.keras.models.load_model('full_verse.keras')

## Tests

In [127]:
# def classify(sentence):
# # sentence = process_review(sentence)
#   sentence = araby.strip_tashkeel(sentence)
#   sequence = [char2idx[char] for char in sentence]
#   sequence = pad_sequences([sequence], maxlen = X_train.shape[1], padding='post', value=0)

#   pred = model.predict(sequence)[0]
#   print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

# def classify(sentence, model_tashkeel, tokenizer):
#     # Remove tashkeel initially
#     sentence = araby.strip_tashkeel(sentence)

#     # Use vocalize_text function to add tashkeel to the sentence
#     sentence = vocalize_text(sentence, model_tashkeel, tokenizer)

#     # Convert the vocalized sentence to sequences
#     sequence = [char2idx.get(char, 0) for char in sentence]
#     sequence = pad_sequences([sequence], maxlen=X_train.shape[1], padding='post', value=0)

#     # Predict the meter class for the sentence
#     pred = model.predict(sequence)[0]
#     print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

def classify(sentence):
    # Convert the sentence to sequences
    sequence = [char2idx.get(char, 0) for char in sentence]
    sequence = pad_sequences([sequence], maxlen=X_train.shape[1], padding='post', value=0)

    # Predict the meter class for the sentence
    pred = model.predict(sequence)[0]
    print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))


In [109]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")
classify("ومن لم يمت بالسيف مات بغيره # تعددت الأسباب والموت واحد")
classify("أنا النبي لا كذب # أنا ابن عبد المطلب")
classify("هذه دراهم اقفرت # أم ربور محتها الدهور")
classify("هزجنا في بواديكم # فأجزلتم عطايانا")
classify("بحر سريع ماله ساحل # مستفعلن مستفعلن فاعلن")
classify("مَا مَضَى فَاتَ وَالْمُؤَمَّلُ غَيْبٌ # وَلَكَ السَّاعَةُ الَّتِيْ أَنْتَ فِيْهَا")
classify("يا ليلُ الصبّ متى غدهُ # أقيامُ الساعة موعدهُ")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
ramal 0.9997578
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
kamel 0.9817662
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
kamel 0.99513716
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
taweel 0.9993303
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
mujtath 0.49932346
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
rajaz 0.7752931
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
hazaj 0.964103
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
rajaz 0.664896
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
khafeef 0.99997747
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
mutadarak 0.9999447


In [96]:
!zip model.zip full_verse.h5

	zip warning: name not matched: full_verse.h5

zip error: Nothing to do! (model.zip)


In [97]:
!ls -l

total 46832
-rw-r--r-- 1 root root  2267882 Nov 12 17:51 baits.zip
-rw-r--r-- 1 root root  2267882 Nov 12 17:57 baits.zip.1
-rw-r--r-- 1 root root  2267882 Nov 12 17:59 baits.zip.2
-rw-r--r-- 1 root root  2267882 Nov 12 18:28 baits.zip.3
drwxr-xr-x 2 root root     4096 Nov 12 18:28 final_baits
-rw-r--r-- 1 root root 38867209 Nov 12 18:37 full_verse.keras
drwxr-xr-x 1 root root     4096 Nov 11 14:21 sample_data


In [98]:
classify("حِصاني كانَ دَلّالَ المَنايا # فَخاضَ غُبارَها وَشَرى وَباعا")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
wafer 0.9576039


In [99]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_path = "Abdou/arabic-tashkeel-flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model_tashkeel = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [100]:
def vocalize_text(text, model_tashkeel, tokenizer, max_length=256, num_beams=4, temperature=0.2, do_sample=False):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_tashkeel.to(device)
    model_tashkeel.eval()
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=max_length, truncation=True, padding="max_length")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        if do_sample:
            outputs = model_tashkeel.generate(
                **inputs,
                max_length=max_length,
                do_sample=True,
                temperature=temperature,
            )
        else:
            outputs = model_tashkeel.generate(
                **inputs,
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True
            )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [102]:
# Quran texts
quran_texts = [
    "قل هو نبأ عظيم أنتم عنه معرضون",
    "إنما يخشى الله من عباده العلماء",
    "والله غالب على أمره ولكن أكثر الناس لا يعلمون",
    "وعلم ءادم الأسماء كلها ثم عرضهم على الملائكة فقال أنبؤوني بأسماء هؤلاء إن كنتم صادقين",
    "وإذ قال موسى لقومه يا قوم لم تؤذونني وقد تعلمون أني رسول الله إليكم",
    "ولله يسجد ما في السماوات وما في الارض من دابة والملاءكة وهم لا يستكبرون",
    "الذي أحسن كل شيء خلقه وبدأ خلق الإنسان من طين",
]
# Hadith texts
hadith_texts = [
    "إن الله لا ينظر إلى صوركم وأموالكم ولكن ينظر إلى قلوبكم وأعمالكم",
    "عن أبي ذر جندب بن جنادة، وأبي عبدالرحمن معاذ بن جبل رضي الله عنهما، عن رسول الله ﷺ، قال: اتق الله حيثما كنت وأتبع السيئة الحسنة تمحها، وخالق الناس بخلق حسن . رواه الترمذي وقال: حديث حسن.",
    "المسلم من سلم المسلمون من لسانه ويده",
    "المؤمن القوي خير وأحب إلى الله من المؤمن الضعيف ، وفي كل خير.",
]
# some Arabic texts
arabic_texts = [
    "إنما الأمم الأخلاق ما بقيت فإن هم ذهبت أخلاقهم ذهبوا",
    "يعد من أكبر علماء الأندلس وأكبر علماء الإسلام تصنيفًا وتأليفًا بعد الطبري، وهو إمام حافظ. فقيه ظاهري، ومجدد القول به، بل محيي المذهب بعد زواله في الشرق. ومتكلم وأديب وشاعر ونسّابة وعالم برجال الحديث وناقد محلل بل وصفه البعض بالفيلسوف كما عد من أوائل من قال بكروية الأرض، كما كان وزير سياسي لبني أمية، سلك طريق نبذ التقليد وتحرير الأتباع، قامت عليه جماعة من المالكية وشـُرد عن وطنه. توفي لاحقاً في منزله في أرض أبويه منت ليشم المعروفة بمونتيخار حالياً، وهي عزبة قريبة من ولبة. وأصل جده يزيد فارسي، أسلم وأول من دخل منهم بلاد المغرب، وكانت بلدهم قرطبة فولد ابن حزم بها في سلخ نهاية رمضان من سنة أربع وثمانين وثلاثمائة."
]

texts = [
    ("Quran Texts", quran_texts),
    ("Hadith Texts", hadith_texts),
    ("Arabic Texts", arabic_texts)
]

for title, texts in texts:
    print(f"=============== {title} ===============")
    for text in texts:
        vocalized_result = vocalize_text(text, model_tashkeel, tokenizer)
        print(f"Input: \n{text}")
        print("="*10)
        print(f"Prediction: \n{vocalized_result}")
        print("="*10)

=============== Quran Texts ===============
Input: 
قل هو نبأ عظيم أنتم عنه معرضون
Prediction: 
قُلْ هُوَ نَبَأٌ عَظِيمٌ أَنْتُمْ عَنْهُ مُعْرِضُونَ
Input: 
إنما يخشى الله من عباده العلماء
Prediction: 
إِنَّمَا يَخْشَى ٱللَّهَ مِنْ عِبَادِهِ ٱلْعُلَمَآءُ
Input: 
والله غالب على أمره ولكن أكثر الناس لا يعلمون
Prediction: 
وَٱللَّهُ غَالِبٌ عَلَىٰٓ أَمْرِهِۦ وَلَٰكِنَّ أَكْثَرَ ٱلنَّاسِ لَا يَعْلَمُونَ
Input: 
وعلم ءادم الأسماء كلها ثم عرضهم على الملائكة فقال أنبؤوني بأسماء هؤلاء إن كنتم صادقين
Prediction: 
وَعَلِّمْ ءَادَمَ ٱلْأَسْمَآءَ كُلَّهَا ثُمَّ عَرَضَهُمْ عَلَى ٱلْمَلاَئِكَةِ فَقَالَ أَنبَؤُونِي بِأَسْمَاءِ هَٰٓؤُلَآءِ إِن كُنتُمْ صَادِقِينَ
Input: 
وإذ قال موسى لقومه يا قوم لم تؤذونني وقد تعلمون أني رسول الله إليكم
Prediction: 
وَإِذْ قَالَ مُوسَىٰ لِقَوْمِهِ يَا قَوْمِ لِمَ تُؤْذُونَنِي وَقَدْ تَعْلَمُونَ أَنِّي رَسُولُ اللَّهِ إِلَيْكُمْ
Input: 
ولله يسجد ما في السماوات وما في الارض من دابة والملاءكة وهم لا يستكبرون
Prediction: 
وَلِلَّهِ يَسْجُدُ مَا فِي السَّمَاوَاتِ وَمَا فِ

In [135]:
sentence = vocalize_text('وصار الوقت جاريًا في غيبتي # يدني النور والظلماء أمامي', model_tashkeel, tokenizer)
sequence = [char2idx.get(char, 0) for char in sentence]
sequence = pad_sequences([sequence], maxlen=X_train.shape[1], padding='post', value=0)
pred = model.predict(sequence)[0]
print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
khafeef 0.7795467


In [129]:
classify(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
khafeef 0.7795467
